In [59]:
!pip install pymongo
import pymongo

In [1]:
!pip install keras
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from keras.models import model_from_json
import json

In [62]:
def load_saved_model_from_db(model_name, client, db, dbconnection):
    json_data = {}
    myclient = pymongo.MongoClient(client)
    
    mydb = myclient[db]
    mycon = mydb[dbconnection]
    data = mycon.find({'created_time':1670573632.139961})
    
    for i in data:
        json_data = i
        
    keras_model = model_from_json(json_data[model_name])
    
    return keras_model

In [12]:
model = model_from_json('{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 18], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_1"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "units": 128, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dropout", "config": {"name": "dropout", "trainable": true, "dtype": "float32", "rate": 0.2, "noise_shape": null, "seed": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 256, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dropout", "config": {"name": "dropout_1", "trainable": true, "dtype": "float32", "rate": 0.4, "noise_shape": null, "seed": null}}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": true, "dtype": "float32", "units": 128, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dropout", "config": {"name": "dropout_2", "trainable": true, "dtype": "float32", "rate": 0.4, "noise_shape": null, "seed": null}}, {"class_name": "Dense", "config": {"name": "dense_3", "trainable": true, "dtype": "float32", "units": 1, "activation": "sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "keras_version": "2.11.0", "backend": "tensorflow"}')

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2432      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1

In [14]:
import tensorflow as tf
opt="adam"
loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
evals=['accuracy']


In [15]:
model.compile(optimizer=opt, loss=loss, metrics=evals) #Use just model.compile() for testing it

In [20]:
import pandas as pd
import time

In [31]:
accuracy = []

In [32]:
start = time.time()
for df in pd.read_csv("SUSY.CSV", chunksize = 1000000):
    X_test = df.iloc[:, 1:].to_numpy()
    Y_test = df.iloc[:, 0].to_numpy()
    score = model.evaluate(X_test, Y_test)
    accuracy.append(score[1])
print(time.time() - start)

31250/31250 [==============================] - 104s 3ms/step - loss: 0.7032 - accuracy: 0.5036
7772.762089729309


In [33]:
len(accuracy) # 75 Million Records with 1M chunksize therefore 70 records

70

In [34]:
final_test_accuracy = sum(accuracy) / len(accuracy)

In [35]:
print("Final test accuracy of the entire dataset: " + str(final_test_accuracy))

Final test accuracy of the entire dataset: 0.5041235029697418
